In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import csv
import numpy as np
import ssl

In [2]:
recipes = pd.read_csv(r'C:\Users\azmeh\Documents\MSc1\VisBio\Projects\Project_Treemap\data_recipe_search.csv')
categories = pd.read_csv(r'C:\Users\azmeh\Documents\MSc1\VisBio\Projects\Project_Treemap\ingredient-categories.csv')

In [3]:
# Reduce recipe data
keepRec = ['name', 'ingredients', 'n_ingredients', 'region', 'country']
recs = recipes[keepRec]

In [15]:
# Reduce cat data
keepCat = ['replaced', 'ingredient_category']
cats2 = categories[keepCat]

# Remove 'replaced' duplicates 
cats2.drop_duplicates(subset=['replaced'], inplace=True)

#set column country to index
cats2 = cats2.set_index('replaced')
#split by comma
cats= cats2.apply(lambda x: x.str.split(',').explode())
#delete square brackets
cats = cats.ingredient_category.str.strip('[]')
#delete  '
cats= cats.str.replace("'", "")
#delete taps
cats = cats.str.strip()

# Store as dataframe
cats = pd.DataFrame(cats)

cats=cats.reset_index()

cats.head(3)

,replaced,ingredient_category
0,lettuce,vegetable
1,french vanilla pudding and pie filling mix,dish
2,stove top stuffing mix,mix


In [16]:
# Get country names in list
allCountries = recs.copy()
allCountries = allCountries['country']
allCountries.drop_duplicates(inplace = True)
allCountries = allCountries.tolist()


# Subset with countries and regions
keepCR = ['region','country']
allCountriesAndRegions = recs[keepCR]
allCountriesAndRegions.drop_duplicates(inplace = True)
allCountriesAndRegions = allCountriesAndRegions.reset_index(drop=True)

allCountriesAndRegions.dropna(inplace=True)


In [17]:
# Get all categories in list
allCats = cats.copy()
allCats = allCats['ingredient_category']
allCats.drop_duplicates(inplace=True)
allCats = allCats.tolist()


In [18]:
# New dataframe containing output df: counts of category occurences
countsCats = pd.DataFrame(allCountriesAndRegions)

# Fill dataframe ith zeros
for x in allCats:
    countsCats[x] = 0




In [19]:
# Get only ingredients
keep= ['country', 'ingredients']
ingredients = recs[keep]
#set column country to index
ingredients = ingredients.set_index('country')
#split by comma
ing= ingredients.apply(lambda x: x.str.split(',').explode())
#delete square brackets
ing = ing.ingredients.str.strip('[]')
#delete  '
ing= ing.str.replace("'", "")
#delete taps
ing = ing.str.strip()

# Store as dataframe
ing = pd.DataFrame(ing)

#get counts of ingredients per country
grouped_list= ing.groupby("country")["ingredients"].value_counts()
#store as df
grouped= pd.DataFrame(grouped_list)
#rename columns
grouped.columns = ['ingredient_counts']
grouped=grouped.reset_index()




In [21]:
# Loop through each country and count ingredients for each category
# Fill dataframe

cRow = 0


while cRow < len(allCountriesAndRegions):

    # Sort into countries
    # New df with only current country ingredients
    currentCountry = countsCats['country'].values[cRow]
    # String of current country
    dfCurrentCountry = grouped[grouped['country']==currentCountry]

    cIngred = 0

    while cIngred < len(dfCurrentCountry):

        # String of current ingredient
        currentIng = dfCurrentCountry['ingredients'].values[cIngred]
        currentIngCounts = dfCurrentCountry['ingredient_counts'].values[cIngred]

        # Find indices of where ingredient exists in category dataframe
        for group in cats: 
            indices = list(np.where(cats["replaced"]==currentIng)[0])

        # Get category name from 'ingredient_category' column of category dataframe
        currentIngCat = cats['ingredient_category'].values[indices]
        currentIngCat = str(currentIngCat)
        currentIngCat = currentIngCat.strip('[]')
        currentIngCat = currentIngCat.replace("'", "")

        # Check for plural, remove 's' from end if plural
        # Skip if category for ingredient does not exist
        if currentIngCat == '':
            if currentIng[len(currentIng)-1] == 's':
                currentIng = currentIng[:-1]

                # String of current ingredient
                currentIng = dfCurrentCountry['ingredients'].values[cIngred]
                currentIngCounts = dfCurrentCountry['ingredient_counts'].values[cIngred]

                # Find indices of where ingredient exists in category dataframe
                for group2 in cats: 
                    indices = list(np.where(cats["replaced"]==currentIng)[0])

            
                if indices != []:
                    currentIngCat = cats['ingredient_category'].values[indices]
                    currentIngCat = str(currentIngCat)
                    currentIngCat = currentIngCat.strip('[]')
                    currentIngCat = currentIngCat.replace("'", "")
                    
                    # Place counts value in relevant category in output df, add onto previous value if exists
                    countsCats.iloc[cRow, countsCats.columns.get_loc(currentIngCat)] = currentIngCounts + countsCats.iloc[cRow, countsCats.columns.get_loc(currentIngCat)]
                    
                else: 
                    # Skip
                    cIngred = cIngred + 1
                    
            else:
                # Skip
                cIngred = cIngred + 1
        else:
            # Place counts value in relevant category in output df, add onto previous value if exists
            countsCats.iloc[cRow, countsCats.columns.get_loc(currentIngCat)] = currentIngCounts + countsCats.iloc[cRow, countsCats.columns.get_loc(currentIngCat)]
            cIngred = cIngred + 1
    cRow = cRow + 1





In [23]:
countsCats.head(3)

,region,country,vegetable,dish,mix,dairy,sauce,herb,confectionery,spread,...,seasoning,vegetarian,seafood,vegan,tools,sweetener,seed,flavoring,fungi,flower
0,central-american,mexican,6997,607,138,5033,2874,2072,173,322,...,1516,231,258,59,108,1117,101,216,36,0
1,north-american,american,28272,3779,2093,32653,9269,11198,1312,2975,...,7853,3653,2234,440,638,17341,1086,1735,447,27
2,north-american,canadian,3037,268,141,3206,736,1351,119,336,...,759,469,219,26,39,2107,166,195,82,5


In [ ]:
# Export dataframe to .csv file
countsCats.to_csv('category_counts.csv', index=False)

In [24]:
# Export to .json file
countsCats.to_json(r'category_counts.json')